In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0227 17:22:46.231209 4556686784 deprecation.py:323] From <ipython-input-1-3c4f46c1a93b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0227 17:22:46.232139 4556686784 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0227 17:22:46.233330 4556686784 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz


W0227 17:22:46.462126 4556686784 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
for folder in ['out/', 'model/']:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [4]:
mb_size = 32
X_dim = mnist.train.images.shape[1]
Z_dim = 16
c_dim_cat = 10
c_dim_con = 2
eps = 1e-8

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])
c_cat = tf.placeholder(tf.float32, shape=[None, c_dim_cat])
c_con = tf.placeholder(tf.float32, shape=[None, c_dim_con])

G_W1 = tf.Variable(xavier_init([Z_dim + c_dim_cat + c_dim_con, 256]))
G_b1 = tf.Variable(tf.zeros(shape=[256]))

G_W2 = tf.Variable(xavier_init([256, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z, c_cat, c_con):
    inputs = tf.concat(axis=1, values=[z, c_cat, c_con])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

Q_W1 = tf.Variable(xavier_init([X_dim, 128]))
Q_b1 = tf.Variable(tf.zeros(shape=[128]))

Q_W2_cat = tf.Variable(xavier_init([128, c_dim_cat]))
Q_b2_cat = tf.Variable(tf.zeros(shape=[c_dim_cat]))

Q_W2_con = tf.Variable(xavier_init([128, c_dim_con]))
Q_b2_con = tf.Variable(tf.zeros(shape=[c_dim_con]))

theta_Q = [Q_W1, Q_W2_cat, Q_W2_con, Q_b1, Q_b2_cat, Q_b2_con]

def Q(x):
    Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    Q_cat_prob = tf.nn.softmax(tf.matmul(Q_h1, Q_W2_cat) + Q_b2_cat)
    Q_con = tf.matmul(Q_h1, Q_W2_con) + Q_b2_con
    return Q_cat_prob, Q_con

def categorical_crossentropy_with_prob(prob, y):
    return tf.reduce_mean(-tf.reduce_sum(y*tf.log(prob + eps), axis=1))

def mean_squared_error(prob, y):
    return tf.reduce_mean(tf.square(y - prob))
#     return tf.losses.mean_squared_error(y, prob)

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def sample_c_cat(m, n):
    return np.random.multinomial(1, n*[0.1], size=m)

def sample_c_con(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

def plot(samples):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z, c_cat, c_con)
D_real = discriminator(X)
D_fake = discriminator(G_sample)
Q_c_cat_given_x, Q_c_con_given_x = Q(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real + eps) + tf.log(1. - D_fake + eps))
G_loss = -tf.reduce_mean(tf.log(D_fake + eps))

Q_loss_cat = categorical_crossentropy_with_prob(Q_c_cat_given_x, c_cat)
Q_loss_con = mean_squared_error(Q_c_con_given_x, c_con)
Q_loss = Q_loss_cat + Q_loss_con

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver = tf.train.AdamOptimizer().minimize(Q_loss, var_list=theta_G + theta_Q)

D_real_predict = tf.cast(tf.greater_equal(D_real, 0.5), tf.float32)
D_real_correct = tf.cast(tf.equal(D_real_predict, np.ones_like([mb_size, 1])), tf.float32)
D_real_accuracy = tf.reduce_mean(D_real_correct)

D_fake_predict = tf.cast(tf.greater_equal(D_fake, 0.5), tf.float32)
D_fake_correct = tf.cast(tf.equal(D_fake_predict, np.zeros_like([mb_size, 1])), tf.float32)
D_fake_accuracy = tf.reduce_mean(D_fake_correct)

D_accuracy = 0.5 * (D_real_accuracy + D_fake_accuracy)

G_predict = tf.cast(tf.greater_equal(D_fake, 0.5), tf.float32)
G_correct = tf.cast(tf.equal(G_predict, np.ones_like([mb_size, 1])), tf.float32)
G_accuracy = tf.reduce_mean(G_correct)

W0227 17:22:46.889415 4556686784 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.restore(sess, "model/save_net.ckpt")

W0227 17:22:47.334863 4556686784 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [ ]:
length = 8
x_lins = np.linspace(-1.5, 1.5, length)
y_lins = np.linspace(-1.5, 1.5, length)
data_draw = np.vstack([np.hstack((x_lins[:,np.newaxis], np.ones([length, 1])*y_lins[length-1-i])) for i in range(length)])

i=0
for it in range(1000000):    
    X_mb, _ =  mnist.train.next_batch(mb_size)
#     Z_sample = sample_Z(mb_size, Z_dim)
    Z_sample = np.zeros((mb_size, Z_dim))
    c_sample_cat = sample_c_cat(mb_size, c_dim_cat)
    c_sample_con = sample_c_con(mb_size, c_dim_con)
        
    _, D_l = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
    _, G_l = sess.run([G_solver, G_loss], feed_dict={Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
    _, Q_l = sess.run([Q_solver, Q_loss], feed_dict={Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
    
    if it%1000==0:
        D_a = sess.run([D_accuracy], feed_dict={X:X_mb, Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
        G_a = sess.run([G_accuracy], feed_dict={Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
    
        print('iter:{}'.format(it))
        print('[D loss:{:.4} acc:{:.4}] [G loss:{:.4} acc:{:.4}] [Q loss:{:.4}]'.format(D_l, D_a[0], G_l, G_a[0], Q_l))
        print()
        
    if it%20000==0:
        for j in range(10):
            n_sample = length*length

            Z_sample = sample_Z(n_sample, Z_dim)
            c_sample_cat = np.zeros([n_sample, c_dim_cat])
            c_sample_cat[:, j] = 1
            
            c_sample_con = data_draw
            
            samples = sess.run(G_sample, feed_dict={Z:Z_sample, c_cat:c_sample_cat, c_con:c_sample_con})
            fig = plot(samples)
            plt.savefig('out/{}.png'.format((str(i)+'_'+str(j)).zfill(3)), bbox_inches='tight')
            plt.close(fig)
        i+=1
        
    if it%1000==0:  
        save_path = saver.save(sess, "model/save_net.ckpt")

iter:0
[D loss:0.4742 acc:0.9219] [G loss:3.494 acc:0.0] [Q loss:0.002363]

iter:1000
[D loss:0.2754 acc:0.9844] [G loss:3.044 acc:0.0] [Q loss:0.002527]

iter:2000
[D loss:0.5205 acc:0.9062] [G loss:2.105 acc:0.09375] [Q loss:0.00296]

iter:3000
[D loss:1.042 acc:0.7969] [G loss:2.212 acc:0.0625] [Q loss:0.003496]

iter:4000
[D loss:0.438 acc:0.9375] [G loss:2.457 acc:0.0625] [Q loss:0.003589]

iter:5000
[D loss:0.8396 acc:0.8281] [G loss:2.155 acc:0.0625] [Q loss:0.001597]

iter:6000
[D loss:0.3867 acc:0.9531] [G loss:2.221 acc:0.0] [Q loss:0.00193]

iter:7000
[D loss:0.5587 acc:0.9219] [G loss:2.922 acc:0.0] [Q loss:0.00225]

iter:8000
[D loss:0.4622 acc:0.9062] [G loss:2.671 acc:0.0] [Q loss:0.002473]

iter:9000
[D loss:0.4512 acc:0.9375] [G loss:2.842 acc:0.03125] [Q loss:0.004488]

iter:10000
[D loss:0.1601 acc:0.9844] [G loss:2.967 acc:0.0] [Q loss:0.002819]

iter:11000
[D loss:0.588 acc:0.9219] [G loss:1.987 acc:0.09375] [Q loss:0.002176]

iter:12000
[D loss:0.7542 acc:0.8906] 

iter:101000
[D loss:0.2995 acc:0.9688] [G loss:2.205 acc:0.03125] [Q loss:0.002272]

iter:102000
[D loss:0.2604 acc:0.9844] [G loss:3.385 acc:0.0] [Q loss:0.002726]

iter:103000
[D loss:0.2022 acc:0.9688] [G loss:3.559 acc:0.0] [Q loss:0.002849]

iter:104000
[D loss:0.3345 acc:0.9844] [G loss:2.842 acc:0.0] [Q loss:0.002703]

iter:105000
[D loss:0.4307 acc:0.8906] [G loss:3.643 acc:0.0] [Q loss:0.002921]

iter:106000
[D loss:0.1951 acc:0.9688] [G loss:3.469 acc:0.0] [Q loss:0.003464]

iter:107000
[D loss:0.5329 acc:0.9062] [G loss:2.235 acc:0.03125] [Q loss:0.0055]

iter:108000
[D loss:0.3877 acc:0.9375] [G loss:3.006 acc:0.03125] [Q loss:0.007164]

iter:109000
[D loss:0.3462 acc:0.9219] [G loss:3.21 acc:0.03125] [Q loss:0.003457]

iter:110000
[D loss:0.259 acc:0.9688] [G loss:3.05 acc:0.0] [Q loss:0.004779]

iter:111000
[D loss:0.579 acc:0.9219] [G loss:3.695 acc:0.0] [Q loss:0.003351]

iter:112000
[D loss:0.3494 acc:0.9062] [G loss:3.973 acc:0.03125] [Q loss:0.003769]

iter:113000
[D